In [60]:
from keras.models import load_model
from keras import backend as K
from keras.losses import binary_crossentropy
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from skimage.morphology import opening, square
from skimage.morphology import remove_small_objects
from scipy import ndimage
import base64
from pycocotools import _mask as coco_mask
import typing as t
import zlib
import pandas as pd


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load preprocessed images
with open('/content/drive/MyDrive/Kaggle/HuBMAP - Hacking the Human Vasculature/Dataset/preprocessed_images.pkl', 'rb') as f:
    preprocessed_images = pickle.load(f)

In [4]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)
def combined_dice_bce_loss(y_true, y_pred, weight=0.5):
    dice_loss_value = dice_loss(y_true, y_pred)
    bce_loss_value = binary_crossentropy(y_true, y_pred)
    return weight * bce_loss_value + (1 - weight) * dice_loss_value

In [5]:
# Load your trained model
model = load_model('/content/drive/MyDrive/Kaggle/HuBMAP - Hacking the Human Vasculature/model1.h5', 
                   custom_objects={'combined_dice_bce_loss': combined_dice_bce_loss, 'dice_coef': dice_coef})

In [6]:
# Get your image ids
image_ids = list(preprocessed_images.keys())

In [7]:
# Get the preprocessed images as a list
images_list = list(preprocessed_images.values())

# Convert the list to a numpy array
images_array = np.array(images_list)

# normalize the images
images_array = images_array/255

In [21]:
print(images_array[0].shape)

(512, 512, 3)


In [58]:
def process_mask(predicted_mask, selem_size=3, min_size=100):
    """
    Perform morphological operations and small object removal on the predicted mask.

    Args:
    predicted_mask (numpy.ndarray): The predicted mask.
    selem_size (int, optional): The size of the structure element for morphological operations. Default is 3.
    min_size (int, optional): The minimum size of the objects to be retained. Default is 100.

    Returns:
    final_mask (numpy.ndarray): The processed mask.
    """

    # Ensure that the input is a 2D array
    if len(predicted_mask.shape) > 2:
        predicted_mask = predicted_mask.squeeze()
    # Create structure element
    selem = square(selem_size)

    # Perform morphological operation (opening)
    opened_mask = opening(predicted_mask, selem)

    # Label connected components
    labeled_array, num_features = ndimage.label(opened_mask)

    # Create an empty mask to hold the remaining components
    final_mask = np.zeros_like(opened_mask)

    for i in range(1, num_features + 1):
        component_size = np.sum(labeled_array == i)
        if component_size >= min_size:
            final_mask += (labeled_array == i)


    return opened_mask

In [9]:
def encode_binary_mask(mask: np.ndarray) -> t.Text:
    """Converts a binary mask into OID challenge encoding ascii text."""

    # check input mask --
    if mask.dtype != np.bool:
      raise ValueError(
          "encode_binary_mask expects a binary mask, received dtype == %s" %
          mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
      raise ValueError(
          "encode_binary_mask expects a 2d mask, received shape == %s" %
          mask.shape)

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str

In [61]:
# Initialize an empty list to hold your submission data
submission_data = []
# For each test image
for idx, image_id in enumerate(image_ids):
    # Break the loop after processing the first 10 images
    if idx >= 10:
        break

    # Generate the predicted mask
    image = np.expand_dims(images_array[idx], axis=0)  # expand dimensions for model
    predicted_mask = model.predict(image)

    # Cast the mask to bool
    binary_predictions = (predicted_mask > 0.5).astype(int)
    predicted_mask =  binary_predictions.squeeze()

    # Post-process the predicted mask
    process_predicted_mask = process_mask(predicted_mask)
    
    # Convert the predicted mask to binary
    binary_predicted_mask = np.where(process_predicted_mask > 0, 1, 0)

    # Compute the confidence score for the prediction
    confidence = 1.0  # Or some other value

    # Cast the mask to bool
    binary_predicted_mask_bool = binary_predicted_mask.astype(bool)
    # Encode the mask
    encoded_mask = encode_binary_mask(binary_predicted_mask_bool)

    # Prepare the prediction string
    prediction_string = f'0 {confidence} {encoded_mask.decode()}'

    # Append this image's data to the submission data
    submission_data.append([image_id, predicted_mask.shape[0], predicted_mask.shape[1], prediction_string])

# Create a DataFrame from the submission data
submission_df = pd.DataFrame(submission_data, columns=['id', 'height', 'width', 'prediction_string'])

# Write the DataFrame to a CSV file for submission
submission_df.to_csv('submission.csv', index=False)

1/1 [==============================] - 5s 5s/step


<ipython-input-9-b565e9965118>:5: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if mask.dtype != np.bool:


1/1 [==============================] - 5s 5s/step


In [ ]:
# Initialize an empty list to hold your submission data
submission_data = []
# For each test image
for idx, image_id in enumerate(image_ids):

    # Generate the predicted mask
    image = np.expand_dims(images_array[idx], axis=0)  # expand dimensions for model
    predicted_mask = model.predict(image)

    # Cast the mask to bool
    binary_predictions = (predicted_mask > 0.5).astype(int)
    predicted_mask =  binary_predictions.squeeze()

    # Post-process the predicted mask
    process_predicted_mask = process_mask(predicted_mask)
    
    # Convert the predicted mask to binary
    binary_predicted_mask = np.where(process_predicted_mask > 0, 1, 0)

    # Compute the confidence score for the prediction
    confidence = 1.0  # Or some other value

    # Cast the mask to bool
    binary_predicted_mask_bool = binary_predicted_mask.astype(bool)
    # Encode the mask
    encoded_mask = encode_binary_mask(binary_predicted_mask_bool)

    # Prepare the prediction string
    prediction_string = f'0 {confidence} {encoded_mask.decode()}'

    # Append this image's data to the submission data
    submission_data.append([image_id, predicted_mask.shape[0], predicted_mask.shape[1], prediction_string])

# Create a DataFrame from the submission data
submission_df = pd.DataFrame(submission_data, columns=['id', 'height', 'width', 'prediction_string'])

# Write the DataFrame to a CSV file for submission
submission_df.to_csv('submission.csv', index=False)